In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import os
import string
from PIL import Image

Copy here the definition of the 'LetterClassifier' class that you defined in 'DL_Lab1_part3.ipynb'.

In [2]:
# Creating a Fully Connected Neural Network Architeture Class 
class LetterClassifier(nn.Module):
    def __init__(self, input_size: int, hidden_layer_size: int, output_size: int):
        super(LetterClassifier, self).__init__()
        # define the network fully connected layers
        self.fc_layer1 = nn.Linear(input_size, 392)
        self.fc_layer2 = nn.Linear(392, 196)
        self.fc_layer3 = nn.Linear(196, output_size)
        # define a flatten layer
        self.flatten = nn.Flatten()
        # define a sigmoid (activation) layer
        self.activation = nn.Sigmoid()
      
    def forward(self, x):
        # define the input layer, operating on flattaned inputs
        flattened_x = self.flatten(x)
        # define the first layer, using linear operation and then activation
        z1 = self.fc_layer1(flattened_x)
        z2 = self.activation(z1)
        z2 = self.fc_layer2(z2)
        z3 = self.activation(z2)
        # define the output layer
        return self.fc_layer3(z3)

Complete the model definition (the call to 'LetterClassifier') with the layer sizes you used for your network.

In [3]:
def load_and_classify(code_file, train_mean, train_std, model_path="letter_classifier.pth"):

    # Check if model file exists
    if not os.path.exists(model_path):
        print(f"Error: Model file {model_path} not found!")
        print("Please make sure you have trained and saved your model first.")
        return None

    # model file exists, load model
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = LetterClassifier(input_size=784, hidden_layer_size=392, output_size=26).to(device)
    model.load_state_dict(torch.load(model_path))
    
    # Load the saved tensor
    test_images = torch.load(code_file)
    
    # Move to the correct device
    test_images = test_images.to(device)
         
    # Normalize the test images
    test_images = (test_images - train_mean) / train_std
    
    # Set model to evaluation mode
    model.eval()
    
    # Get predictions
    with torch.no_grad():
        outputs = model(test_images)
        _, predicted_indices = torch.max(outputs, 1)
    
    # Convert class indices back to letters (0->A, 1->B, etc.)
    predicted_letters = [chr(idx.item() + ord('A')) for idx in predicted_indices]
    predicted_code = ''.join(predicted_letters)
    
    print(f"Predicted code: {predicted_code}")

    if 0:
        # Visualize with predictions
        fig, axes = plt.subplots(1, 5, figsize=(15, 3))
        for i, img in enumerate(test_images.cpu()):
            # Transpose the image before displaying
            img_display = img.numpy().T
            axes[i].imshow(img_display, cmap='gray')
            axes[i].set_title(f"Predicted: {predicted_letters[i]}")
            axes[i].axis('off')
        plt.tight_layout()
        plt.show()
    
    return predicted_code

Copy here the normalization values that were saved in 'norm_values.txt', and the name of the 'bin' file that holds the code's data.

In [4]:
# complete the code file according to your chose :-)
# red_lock_code.bin / black_lock_code.bin / blue_lock_code.bin / grey_lock_code.bin
code_file = "black_lock_code.bin"
mean_value = 43.92
std_value = 84.39
model_file = "letter_classifier.pth"
if os.path.exists(code_file) and os.path.exists(model_file):
    decoded_code = load_and_classify(code_file, float(mean_value), float(std_value), model_file)
    if decoded_code:
        print(f"Try using {decoded_code.upper()} and see if you solved the challenge!")
else:
    print(f"File {code_file} or {model_file} not found!")

Predicted code: TANSU
Try using TANSU and see if you solved the challenge!
